In [1]:
import Nets
using CoordinateTransformations
using ForwardDiff
using MLDataPattern
using Plots; gr()
using ProgressMeter

INFO: Precompiling module GR.


In [61]:
f(x) = [sin(2 * x[1])]

X = [[x] for x in linspace(-pi, pi, 60)]
Y = [hcat(f(x), ForwardDiff.jacobian(f, x)) for x in X];

data = collect(zip(X, Y))
srand(1)
train_data, test_data = splitobs(shuffleobs(data), 1/10);

In [62]:
widths = [1, 16, 16, 16, 1]
activation = Nets.gaussian

function sensitive_loss(λ)
    q = [1.0-λ λ]
    (params, x, y) -> begin
            sum(abs2, q .* (Nets.predict_sensitivity(Nets.Net(Nets.Params(widths, params), activation), x) .- y))
    end
end

sensitive_loss (generic function with 1 method)

In [63]:
start_params = 0.2 * randn(Nets.Params{Float64}, widths).data
nepoch = 100;

In [119]:
params = copy(start_params)
net = Nets.Net(Nets.Params(widths, params), activation)

train_loss = sensitive_loss(0.5)
validate_loss = params -> mean(xy -> sensitive_loss(0)(params, xy[1], xy[2]), train_data)
test_loss = params -> mean(xy -> sensitive_loss(0)(params, xy[1], xy[2]), test_data)

train_losses = [validate_loss(params)]
test_losses = [test_loss(params)]

@showprogress for i in 1:nepoch
    Nets.adam!(train_loss, params, train_data, Nets.AdamOpts(learning_rate=0.005 * 0.99^i, batch_size=2))
    push!(train_losses, validate_loss(params))
    push!(test_losses, test_loss(params))
end


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [120]:
plt = plot(train_losses, ylim=(0, 1), xlabel="training iteration", ylabel="loss", label="training loss")
plot!(plt, test_losses, label="test loss")
savefig(plt, "losses_0.5.svg")
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 training iteration 
 
 
 loss 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,37.5754 44.8976,83.7061 50.4252,191.916 55.9528,223.792 61.4803,216.927 67.0079,214.697 72.5354,224.433 78.063,223.043 83.5906,208.086 89.1181,206.096 
 94.6457,226.011 100.173,232.273 105.701,217.79 111.228,237.657 116.756,238.181 122.283,238.537 127.811,232.206 133.339,214.056 138.866,236.68 144.394,248.971 
 149.921,244.23 155.449,253.608 160.976,250.024 166.504,247.078 172.031,253.2 177.559,250.461 183.087,250.703 188.614,235.835 194.142,219.015 199.669,245.196 
 205.197,244.837 210.724,240.094 216.252,242.34 221.78,252.149 227.307,244.457 232.835,241.184 238.362,242.322 243.89,246.767 249.417,243.355 254.945,269.491 
 260.472,267.989 266,260.794 271.528,268.547 277.055,275.504 282.583,288.808 288.11,286.931 293.638,290.097 299.165,302.928 304.693,308.947 310.22,306.93 
 315.748,303.816 321.276,322.204 326.803,320.721 332.331,325.553 337.858,330.565 343.386,332.157 348.913,334.395 354.441,340.293 359.969,345.886 365.496,342.803 
 371.024,339.888 376.551,353.309 382.079,351.08 387.606,352.393 393.134,357.862 398.661,356.025 404.189,356.451 409.717,360.059 415.244,355.655 420.772,359.348 
 426.299,360.63 431.827,351.28 437.354,360.163 442.882,350.841 448.409,358.098 453.937,351.112 459.465,363.097 464.992,358.343 470.52,363.22 476.047,362.682 
 481.575,362.689 487.102,362.933 492.63,360.912 498.157,362.589 503.685,360.81 509.213,364.34 514.74,363.08 520.268,366.191 525.795,365.644 531.323,362.878 
 536.85,364.037 542.378,358.145 547.906,366.823 553.433,359.893 558.961,365.019 564.488,358.888 570.016,366.292 575.543,361.283 581.071,365.791 586.598,364.802 
 592.126,366.201 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,70.0401 44.8976,105.928 50.4252,181.998 55.9528,188.404 61.4803,165.441 67.0079,161.917 72.5354,179.266 78.063,171.956 83.5906,149.661 89.1181,146.274 
 94.6457,192.681 100.173,196.704 105.701,193.151 111.228,194.552 116.756,191.62 122.283,188.75 127.811,183.245 133.339,162.047 138.866,189.298 144.394,206 
 149.921,198.89 155.449,206.199 160.976,199.907 166.504,201.085 172.031,210.087 177.559,208.617 183.087,206.671 188.614,202.546 194.142,195.174 199.669,215.785 
 205.197,209.014 210.724,208.558 216.252,216.936 221.78,212.537 227.307,207.219 232.835,211.396 238.362,223.419 243.89,219.167 249.417,232.269 254.945,249.147 
 260.472,249.799 266,246.094 271.528,232.701 277.055,258.211 282.583,259.564 288.11,248.074 293.638,269.351 299.165,276.482 304.693,276.437 310.22,283.874 
 315.748,278.745 321.276,307.744 326.803,303.716 332.331,306.551 337.858,320.132 343.386,325.129 348.913,330.237 354.441,332.269 359.969,337.478 365.496,339.46 
 371.024,337.559 376.551,343.86 382.079,347.907 387.606,348.455 393.134,350.406 398.661,354.212 404.189,351.743 409.717,354.754 415.244,355.232 420.772,356.007 
 426.299,357.01 431.827,352.446 437.354,358.656 442.882,353.68 448.409,359.018 453.937,353.925 459.465,361.07 464.992,359.29 470.52,362.431 476.047,362.317 
 481.575,361.222 487.102,361.006 492.63,361.246 498.157,361.089 503.685,356.546 509.213,362.557 514.74,363.586 520.268,363.088 525.795,366.465 531.323,360.803 
 536.85,365.211 542.378,360.096 547.906,365.605 553.433,359.892 558.961,365.683 564.488,359.934 570.016,366.371 575.543,359.408 581.071,366.363 586.598,363.003 
 592.126,366.577 
 "/>
 
 
 
 
 training loss 
 
 
 
 test loss

In [121]:
plt = plot(linspace(-pi, pi, 51), x -> net([x])[1], ylim=(-4, 4), ylabel="f(x)", xlabel="x", label="prediction")
plot!(plt, [x[1] for (x, y) in train_data], [y[1] for (x, y) in train_data], line=nothing, marker=:circle, label="training data")
plot!(plt, linspace(-pi, pi, 101), x -> sin(2x), label="ground truth")
savefig(plt, "reconstruction_0.5.svg")
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 x 
 
 
 f(x) 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.0141,184.012 65.4435,173.428 75.8728,163.174 86.3022,154.324 96.7315,147.642 107.161,143.529 117.59,142.099 128.02,143.314 138.449,147.069 148.878,153.192 
 159.308,161.396 169.737,171.216 180.166,182.013 190.596,193.038 201.025,203.536 211.454,212.834 221.884,220.399 232.313,225.851 242.743,228.948 253.172,229.541 
 263.601,227.526 274.031,222.882 284.46,215.798 294.889,206.797 305.319,196.703 315.748,186.43 326.177,176.764 336.607,168.257 347.036,161.244 357.465,155.905 
 367.895,152.321 378.324,150.524 388.754,150.537 399.183,152.405 409.612,156.203 420.042,162.02 430.471,169.889 440.9,179.68 451.33,190.97 461.759,202.956 
 472.188,214.47 482.618,224.146 493.047,230.73 503.476,233.422 513.906,232.119 524.335,227.425 534.765,220.413 545.194,212.258 555.623,203.939 566.053,196.109 
 576.482,189.11 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.0141,188.189 60.2288,182.539 65.4435,176.978 70.6581,171.594 75.8728,166.472 81.0875,161.692 86.3022,157.33 91.5169,153.455 96.7315,150.128 101.946,147.401 
 107.161,145.317 112.376,143.909 117.59,143.199 122.805,143.199 128.02,143.909 133.234,145.317 138.449,147.401 143.664,150.128 148.878,153.455 154.093,157.33 
 159.308,161.692 164.522,166.472 169.737,171.594 174.952,176.978 180.166,182.539 185.381,188.189 190.596,193.839 195.81,199.4 201.025,204.784 206.24,209.906 
 211.454,214.686 216.669,219.047 221.884,222.923 227.098,226.25 232.313,228.977 237.528,231.061 242.743,232.469 247.957,233.179 253.172,233.179 258.387,232.469 
 263.601,231.061 268.816,228.977 274.031,226.25 279.245,222.923 284.46,219.047 289.675,214.686 294.889,209.906 300.104,204.784 305.319,199.4 310.533,193.839 
 315.748,188.189 320.963,182.539 326.177,176.978 331.392,171.594 336.607,166.472 341.821,161.692 347.036,157.33 352.251,153.455 357.465,150.128 362.68,147.401 
 367.895,145.317 373.109,143.909 378.324,143.199 383.539,143.199 388.754,143.909 393.968,145.317 399.183,147.401 404.398,150.128 409.612,153.455 414.827,157.33 
 420.042,161.692 425.256,166.472 430.471,171.594 435.686,176.978 440.9,182.539 446.115,188.189 451.33,193.839 456.544,199.4 461.759,204.784 466.974,209.906 
 472.188,214.686 477.403,219.047 482.618,222.923 487.832,226.25 493.047,228.977 498.262,231.061 503.476,232.469 508.691,233.179 513.906,233.179 519.12,232.469 
 524.335,231.061 529.55,228.977 534.765,226.25 539.979,222.923 545.194,219.047 550.409,214.686 555.623,209.906 560.838,204.784 566.053,199.4 571.267,193.839 
 576.482,188.189 
 "/>
 
 
 
 
 prediction 
 
 
 
 
 
 training data 
 
 
 
 ground truth

In [122]:
params = copy(start_params)
net = Nets.Net(Nets.Params(widths, params), activation)

train_loss = sensitive_loss(0.)
validate_loss = params -> mean(xy -> sensitive_loss(0)(params, xy[1], xy[2]), train_data)
test_loss = params -> mean(xy -> sensitive_loss(0)(params, xy[1], xy[2]), test_data)

train_losses = [validate_loss(params)]
test_losses = [test_loss(params)]

@showprogress for i in 1:nepoch
    Nets.adam!(train_loss, params, train_data, Nets.AdamOpts(learning_rate=0.01 * 0.99^i, batch_size=2))
    push!(train_losses, validate_loss(params))
    push!(test_losses, test_loss(params))
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [123]:
plt = plot(train_losses, ylim=(0, 1), xlabel="training iteration", ylabel="loss", label="training loss")
plot!(plt, test_losses, label="test loss")
savefig(plt, "losses_0.0.svg")
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 training iteration 
 
 
 loss 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,37.5754 44.8976,224.132 50.4252,232.463 55.9528,242.652 61.4803,203.533 67.0079,247.675 72.5354,263.063 78.063,265.718 83.5906,267.961 89.1181,269.326 
 94.6457,249.588 100.173,265.612 105.701,292.204 111.228,295.647 116.756,302.794 122.283,301.996 127.811,303.369 133.339,259.5 138.866,297.081 144.394,301.231 
 149.921,309.358 155.449,274.441 160.976,311.35 166.504,310.427 172.031,318.943 177.559,317.69 183.087,319.166 188.614,315.495 194.142,317.885 199.669,308.123 
 205.197,322.673 210.724,322.92 216.252,320.157 221.78,317.531 227.307,320.839 232.835,315.267 238.362,311.737 243.89,321.572 249.417,324.193 254.945,314.518 
 260.472,322.986 266,315.697 271.528,321.166 277.055,326.298 282.583,327.393 288.11,327.775 293.638,318.214 299.165,328.254 304.693,331.193 310.22,328.508 
 315.748,326.647 321.276,327.828 326.803,323.384 332.331,329.452 337.858,330.279 343.386,330.747 348.913,316.282 354.441,331.646 359.969,327.402 365.496,335.143 
 371.024,334.613 376.551,336.767 382.079,328.916 387.606,330.489 393.134,328.136 398.661,328.241 404.189,338.149 409.717,338.987 415.244,333.209 420.772,341.456 
 426.299,335.304 431.827,339.31 437.354,342.232 442.882,340.389 448.409,342.461 453.937,342.244 459.465,336.336 464.992,346.405 470.52,338.643 476.047,347.231 
 481.575,347.108 487.102,341.96 492.63,346.534 498.157,349.031 503.685,351.343 509.213,351.7 514.74,352.212 520.268,348.628 525.795,352.527 531.323,353.567 
 536.85,351.079 542.378,353.471 547.906,348.853 553.433,349.712 558.961,352.423 564.488,347.251 570.016,347.54 575.543,353.858 581.071,347.159 586.598,352.996 
 592.126,353.886 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,70.0401 44.8976,189.103 50.4252,164.65 55.9528,197.041 61.4803,112.345 67.0079,196.977 72.5354,187.209 78.063,196.611 83.5906,195.607 89.1181,169.555 
 94.6457,186.729 100.173,191.222 105.701,183.317 111.228,179.468 116.756,170.573 122.283,176.563 127.811,172.504 133.339,165.339 138.866,158.148 144.394,179.029 
 149.921,171.952 155.449,117.075 160.976,158.885 166.504,129.072 172.031,171.09 177.559,143.642 183.087,147.868 188.614,164.546 194.142,163.017 199.669,140.389 
 205.197,179.435 210.724,170.878 216.252,187.195 221.78,166.449 227.307,181.636 232.835,194.967 238.362,205.101 243.89,187.689 249.417,160.772 254.945,154.718 
 260.472,195.645 266,182.352 271.528,204.847 277.055,201.57 282.583,175.381 288.11,170.312 293.638,169.809 299.165,164.018 304.693,188.796 310.22,193.616 
 315.748,188.9 321.276,174.116 326.803,166.063 332.331,175.338 337.858,203.044 343.386,172.436 348.913,159.771 354.441,177.05 359.969,146.391 365.496,190.346 
 371.024,213.182 376.551,192.96 382.079,152.461 387.606,154.145 393.134,153.156 398.661,172.198 404.189,215.167 409.717,210.153 415.244,203.19 420.772,231.423 
 426.299,244.538 431.827,230.326 437.354,249.086 442.882,206.567 448.409,213.255 453.937,235.483 459.465,264.039 464.992,220.873 470.52,219.538 476.047,237.548 
 481.575,223.955 487.102,218.391 492.63,249.49 498.157,232.442 503.685,249.637 509.213,269.075 514.74,251.832 520.268,284.851 525.795,250.104 531.323,271.716 
 536.85,267.37 542.378,256.6 547.906,228.796 553.433,252.183 558.961,245.589 564.488,261.945 570.016,289.128 575.543,252.674 581.071,266.162 586.598,267.767 
 592.126,264.351 
 "/>
 
 
 
 
 training loss 
 
 
 
 test loss

In [124]:
plt = plot(linspace(-pi, pi, 51), x -> net([x])[1], ylim=(-4, 4), ylabel="f(x)", xlabel="x", label="prediction")
plot!(plt, [x[1] for (x, y) in train_data], [y[1] for (x, y) in train_data], line=nothing, marker=:circle, label="training data")
plot!(plt, linspace(-pi, pi, 101), x -> sin(2x), label="ground truth")
savefig(plt, "reconstruction_0.0.svg")
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 x 
 
 
 f(x) 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.0141,165.788 65.4435,162.142 75.8728,158.728 86.3022,155.646 96.7315,152.968 107.161,150.738 117.59,148.978 128.02,147.704 138.449,146.948 148.878,146.775 
 159.308,147.318 169.737,148.809 180.166,151.602 190.596,156.152 201.025,162.861 211.454,171.755 221.884,182.132 232.313,192.619 242.743,201.754 253.172,208.609 
 263.601,212.91 274.031,214.805 284.46,214.606 294.889,212.64 305.319,209.233 315.748,204.786 326.177,199.82 336.607,194.904 347.036,190.521 357.465,186.974 
 367.895,184.393 378.324,182.786 388.754,182.107 399.183,182.3 409.612,183.313 420.042,185.101 430.471,187.611 440.9,190.759 451.33,194.408 461.759,198.362 
 472.188,202.374 482.618,206.185 493.047,209.568 503.476,212.366 513.906,214.507 524.335,215.992 534.765,216.875 545.194,217.24 555.623,217.184 566.053,216.804 
 576.482,216.191 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.0141,188.189 60.2288,182.539 65.4435,176.978 70.6581,171.594 75.8728,166.472 81.0875,161.692 86.3022,157.33 91.5169,153.455 96.7315,150.128 101.946,147.401 
 107.161,145.317 112.376,143.909 117.59,143.199 122.805,143.199 128.02,143.909 133.234,145.317 138.449,147.401 143.664,150.128 148.878,153.455 154.093,157.33 
 159.308,161.692 164.522,166.472 169.737,171.594 174.952,176.978 180.166,182.539 185.381,188.189 190.596,193.839 195.81,199.4 201.025,204.784 206.24,209.906 
 211.454,214.686 216.669,219.047 221.884,222.923 227.098,226.25 232.313,228.977 237.528,231.061 242.743,232.469 247.957,233.179 253.172,233.179 258.387,232.469 
 263.601,231.061 268.816,228.977 274.031,226.25 279.245,222.923 284.46,219.047 289.675,214.686 294.889,209.906 300.104,204.784 305.319,199.4 310.533,193.839 
 315.748,188.189 320.963,182.539 326.177,176.978 331.392,171.594 336.607,166.472 341.821,161.692 347.036,157.33 352.251,153.455 357.465,150.128 362.68,147.401 
 367.895,145.317 373.109,143.909 378.324,143.199 383.539,143.199 388.754,143.909 393.968,145.317 399.183,147.401 404.398,150.128 409.612,153.455 414.827,157.33 
 420.042,161.692 425.256,166.472 430.471,171.594 435.686,176.978 440.9,182.539 446.115,188.189 451.33,193.839 456.544,199.4 461.759,204.784 466.974,209.906 
 472.188,214.686 477.403,219.047 482.618,222.923 487.832,226.25 493.047,228.977 498.262,231.061 503.476,232.469 508.691,233.179 513.906,233.179 519.12,232.469 
 524.335,231.061 529.55,228.977 534.765,226.25 539.979,222.923 545.194,219.047 550.409,214.686 555.623,209.906 560.838,204.784 566.053,199.4 571.267,193.839 
 576.482,188.189 
 "/>
 
 
 
 
 prediction 
 
 
 
 
 
 training data 
 
 
 
 ground truth

In [129]:
params = copy(start_params)
net = Nets.Net(Nets.Params(widths, params), activation)

train_loss = sensitive_loss(1.0)
validate_loss = params -> mean(xy -> sensitive_loss(0)(params, xy[1], xy[2]), train_data)
test_loss = params -> mean(xy -> sensitive_loss(0)(params, xy[1], xy[2]), test_data)

train_losses = [validate_loss(params)]
test_losses = [test_loss(params)]

@showprogress for i in 1:nepoch
    Nets.adam!(train_loss, params, train_data, Nets.AdamOpts(learning_rate=0.005 * 0.99^i, batch_size=2))
    push!(train_losses, validate_loss(params))
    push!(test_losses, test_loss(params))
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [130]:
plt = plot(train_losses, ylim=(0, 1), xlabel="training iteration", ylabel="loss", label="training loss")
plot!(plt, test_losses, label="test loss")
savefig(plt, "losses_1.0.svg")
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 training iteration 
 
 
 loss 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,37.5754 44.8976,10.0623 50.4252,-0.154188 55.9528,-18.5611 61.4803,-0.625833 67.0079,-53.2187 72.5354,34.3004 78.063,69.2183 83.5906,81.3143 89.1181,-8.31077 
 94.6457,54.49 100.173,19.2401 105.701,-87.2662 111.228,84.9303 116.756,9.29433 122.283,73.305 127.811,86.0141 133.339,68.5245 138.866,-46.3377 144.394,-32.1519 
 149.921,-51.4599 155.449,29.6445 160.976,48.1356 166.504,93.2258 172.031,21.9812 177.559,13.1435 183.087,78.1344 188.614,25.8865 194.142,75.4362 199.669,20.5255 
 205.197,-6.64145 210.724,52.9693 216.252,59.2622 221.78,30.483 227.307,-17.7734 232.835,24.0827 238.362,13.1467 243.89,36.6009 249.417,11.9884 254.945,43.2565 
 260.472,-64.3439 266,67.9826 271.528,76.0407 277.055,-36.0773 282.583,-35.3674 288.11,-14.1266 293.638,-20 299.165,-29.6524 304.693,-25.2579 310.22,-71.8568 
 315.748,38.2757 321.276,-14.6979 326.803,38.2364 332.331,127.344 337.858,-9.35269 343.386,104.169 348.913,62.2107 354.441,31.1603 359.969,38.3634 365.496,11.2155 
 371.024,136.221 376.551,173.682 382.079,125.388 387.606,133.643 393.134,153.137 398.661,131.508 404.189,155.068 409.717,91.0113 415.244,184.97 420.772,96.4282 
 426.299,205.412 431.827,112.802 437.354,211.204 442.882,124.279 448.409,232.942 453.937,138.771 459.465,159 464.992,144.455 470.52,136.673 476.047,169.333 
 481.575,91.4956 487.102,152.058 492.63,39.942 498.157,167.46 503.685,119.342 509.213,189.008 514.74,139.621 520.268,185.631 525.795,221.597 531.323,186.789 
 536.85,224.378 542.378,216.256 547.906,234.913 553.433,227 558.961,240.337 564.488,224.771 570.016,210.082 575.543,196.596 581.071,154.403 586.598,184.954 
 592.126,173.722 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,70.0401 44.8976,46.2893 50.4252,34.858 55.9528,18.6215 61.4803,32.4742 67.0079,-23.0664 72.5354,56.4713 78.063,84.0608 83.5906,88.083 89.1181,-20.9688 
 94.6457,36.309 100.173,-18.4006 105.701,-150.396 111.228,57.8525 116.756,-51.4135 122.283,28.538 127.811,46.8695 133.339,15.6428 138.866,-150.914 144.394,-147.14 
 149.921,-181.383 155.449,-61.4075 160.976,-26.5487 166.504,15.4931 172.031,-81.8064 177.559,-80.6003 183.087,17.7268 188.614,-74.5557 194.142,-12.676 199.669,-107.621 
 205.197,-155.347 210.724,-68.4471 216.252,-56.7475 221.78,-105.019 227.307,-139.084 232.835,-70.6117 238.362,-112.559 243.89,-65.1197 249.417,-84.7234 254.945,-60.4764 
 260.472,-199.131 266,-29.9618 271.528,-38.9006 277.055,-166.138 282.583,-143.767 288.11,-103.134 293.638,-78.8023 299.165,-82.2071 304.693,-57.2193 310.22,-111.362 
 315.748,30.2125 321.276,-46.9013 326.803,62.0273 332.331,105.579 337.858,-32.8356 343.386,56.2077 348.913,24.6095 354.441,-24.136 359.969,32.9095 365.496,0.753237 
 371.024,80.6365 376.551,163.219 382.079,74.1665 387.606,118.278 393.134,148.591 398.661,86.4307 404.189,165.212 409.717,43.9124 415.244,179.194 420.772,83.67 
 426.299,200.421 431.827,98.3268 437.354,207.335 442.882,114.766 448.409,211.208 453.937,113.284 459.465,162.383 464.992,123.241 470.52,112.257 476.047,173.542 
 481.575,81.3994 487.102,166.604 492.63,50.0308 498.157,175.839 503.685,111.284 509.213,175.601 514.74,158.097 520.268,147.579 525.795,226.058 531.323,156.501 
 536.85,231.042 542.378,183.826 547.906,226.202 553.433,209.164 558.961,234.269 564.488,199.577 570.016,214.329 575.543,183.353 581.071,139.969 586.598,162.033 
 592.126,186.907 
 "/>
 
 
 
 
 training loss 
 
 
 
 test loss

In [131]:
plt = plot(linspace(-pi, pi, 51), x -> net([x])[1], ylim=(-4, 4), ylabel="f(x)", xlabel="x", label="prediction")
plot!(plt, [x[1] for (x, y) in train_data], [y[1] for (x, y) in train_data], line=nothing, marker=:circle, label="training data")
plot!(plt, linspace(-pi, pi, 101), x -> sin(2x), label="ground truth")
savefig(plt, "reconstruction_1.0.svg")
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 x 
 
 
 f(x) 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.0141,166.028 65.4435,155.219 75.8728,145.4 86.3022,137.392 96.7315,131.722 107.161,128.631 117.59,128.132 128.02,130.071 138.449,134.168 148.878,140.036 
 159.308,147.2 169.737,155.111 180.166,163.179 190.596,170.816 201.025,177.48 211.454,182.718 221.884,186.187 232.313,187.665 242.743,187.051 253.172,184.365 
 263.601,179.757 274.031,173.524 284.46,166.104 294.889,158.039 305.319,149.909 315.748,142.258 326.177,135.531 336.607,130.042 347.036,125.985 357.465,123.448 
 367.895,122.443 378.324,122.931 388.754,124.847 399.183,128.125 409.612,132.72 420.042,138.616 430.471,145.797 440.9,154.184 451.33,163.543 461.759,173.412 
 472.188,183.053 482.618,191.469 493.047,197.559 503.476,200.482 513.906,199.974 524.335,196.355 534.765,190.298 545.194,182.568 555.623,173.883 566.053,164.847 
 576.482,155.94 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 55.0141,188.189 60.2288,182.539 65.4435,176.978 70.6581,171.594 75.8728,166.472 81.0875,161.692 86.3022,157.33 91.5169,153.455 96.7315,150.128 101.946,147.401 
 107.161,145.317 112.376,143.909 117.59,143.199 122.805,143.199 128.02,143.909 133.234,145.317 138.449,147.401 143.664,150.128 148.878,153.455 154.093,157.33 
 159.308,161.692 164.522,166.472 169.737,171.594 174.952,176.978 180.166,182.539 185.381,188.189 190.596,193.839 195.81,199.4 201.025,204.784 206.24,209.906 
 211.454,214.686 216.669,219.047 221.884,222.923 227.098,226.25 232.313,228.977 237.528,231.061 242.743,232.469 247.957,233.179 253.172,233.179 258.387,232.469 
 263.601,231.061 268.816,228.977 274.031,226.25 279.245,222.923 284.46,219.047 289.675,214.686 294.889,209.906 300.104,204.784 305.319,199.4 310.533,193.839 
 315.748,188.189 320.963,182.539 326.177,176.978 331.392,171.594 336.607,166.472 341.821,161.692 347.036,157.33 352.251,153.455 357.465,150.128 362.68,147.401 
 367.895,145.317 373.109,143.909 378.324,143.199 383.539,143.199 388.754,143.909 393.968,145.317 399.183,147.401 404.398,150.128 409.612,153.455 414.827,157.33 
 420.042,161.692 425.256,166.472 430.471,171.594 435.686,176.978 440.9,182.539 446.115,188.189 451.33,193.839 456.544,199.4 461.759,204.784 466.974,209.906 
 472.188,214.686 477.403,219.047 482.618,222.923 487.832,226.25 493.047,228.977 498.262,231.061 503.476,232.469 508.691,233.179 513.906,233.179 519.12,232.469 
 524.335,231.061 529.55,228.977 534.765,226.25 539.979,222.923 545.194,219.047 550.409,214.686 555.623,209.906 560.838,204.784 566.053,199.4 571.267,193.839 
 576.482,188.189 
 "/>
 
 
 
 
 prediction 
 
 
 
 
 
 training data 
 
 
 
 ground truth